# Introduction {#sec-introduction}

In [1]:
#| label: setup
import matplotlib.pyplot as plt
import pandas as pd
!pip install linearmodels
from linearmodels.panel import PanelOLS


- lnaction_nonoil es el logaritmo de las acciones regulatorias sobre empresas no relacionadas con el petróleo/gas
- lnest_nonoil es el logaritmo de la cantidad de establecimientos regulados
- lnformal_nonoil el logaritmo de la cantidad de acciones regulatorias formales
- fracked es un indicador que toma el valor de 1 si un zipcode registró algún pozo de fracking durante la muestra y 0 en otro caso
- treatment es una variable que toma el valor de 1 a partir de 2005 y 0 en otro caso
- lnestablishments el logaritmo de la cantidad de establecimiento privados 
- lnemployment logaritmo de empleados a nivel del county

xtset zipcode year

*Genero la variable con la interacción

gen frack_post= fracked * treatment

*Regresiones para replicar tabla 1 de Gonzales (2024)

eststo actions: xtreg lnactionnonoil frack_post fracked treatment i.year, fe cluster(zipcode)

eststo actions_c: xtreg lnactionnonoil frack_post fracked treatment lnestab lnemp i.year, fe cluster(zipcode)

eststo facilities: xtreg lnone_non_oil frack_post fracked treatment i.year, fe cluster(zipcode)

eststo facilities_c: xtreg lnone_non_oil frack_post fracked treatment lnestab lnemp i.year, fe cluster(zipcode)

eststo formal: xtreg lnstate_forma~l  frack_post fracked treatment lnestab lnemp i.year, fe cluster(zipcode)

eststo formal_c: xtreg lnstate_forma~l  frack_post fracked treatment lnestab lnemp i.year, fe cluster(zipcode)

*Tabla con resultados, fue formateada en LaTex para la entrega.
est table actions actions_c facilities facilities_c formal formal_c, star(0.1 0.05 0.01) b(%9.4f) stats(N r2 r2_w) drop(_cons)


$$
\log Action Non Oil_{it} =  
$$

In [46]:
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [57]:
df['actinonnoil'] = np.exp(df['lnactionnonoil'])

In [2]:
df = pd.read_stata('/Users/mcargnel/Documents/mea/tesis/data/zc_level.dta')

In [3]:
df['frack_post'] = df['fracked'] * df['treatment']
df_panel = df.set_index(['zipcode', 'year'])

In [60]:
!pip install doubleml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 60.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [doubleml]1/2 [doubleml]


In [5]:
df_panel_2 = df_panel.reset_index().dropna()

In [ ]:
# Option 1: Using EconML (most popular)
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Prepare data
y = df_panel_2['lnactionnonoil']
T = df_panel_2['frack_post']  # treatment
X = df_panel_2[['fracked', 'treatment', 'lnestab', 'lnemp']]  # controls

# Add fixed effects as controls
entity_dummies = pd.get_dummies(df_panel_2['zipcode'], prefix='entity')
time_dummies = pd.get_dummies(df_panel_2['year'], prefix='year')
X_with_fe = pd.concat([X, entity_dummies, time_dummies], axis=1)

# Fit DML model
dml = LinearDML(
    model_y=RandomForestRegressor(n_estimators=100),
    model_t=RandomForestRegressor(n_estimators=100),
    discrete_treatment=False,
    cv=5
)

dml.fit(Y=y, T=T, X=X_with_fe)

# Results
print("DML Treatment Effect:", dml.coef_[0])
print("Confidence Interval:", dml.coef__interval(alpha=0.05))
print("Standard Error:", dml.coef__stderr)

# =================================================================

In [ ]:


# Option 2: Using DoubleML (R-style interface)
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor

# Create DoubleML data object
dml_data = DoubleMLData(
    data=df_panel.reset_index(),
    y_col='lnactionnonoil',
    d_cols='frack_post',
    x_cols=['fracked', 'treatment', 'lnestab', 'lnemp']
)

# Initialize model
dml_plr = DoubleMLPLR(
    obj_dml_data=dml_data,
    ml_g=RandomForestRegressor(n_estimators=100),
    ml_m=RandomForestRegressor(n_estimators=100),
    n_folds=5
)

# Fit and get results
dml_plr.fit()
print("\nDoubleML Results:")
print(dml_plr.summary)

In [43]:
# Model 1: actions_c - lnactionnonoil with controls
print("=== Model 1: actions_c ===")
model1 = PanelOLS(
    dependent=df_panel['lnactionnonoil'],
    exog=df_panel[['frack_post', 'fracked', 'treatment', 'lnestab', 'lnemp']],
    entity_effects=True,  # zipcode fixed effects
    time_effects=True,    # year fixed effects (i.year)
    drop_absorbed=True
)
results['actions_c'] = model1.fit(cov_type='clustered', cluster_entity=True)
print(results['actions_c'].summary)


=== Model 1: actions_c ===


/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_53495/3964689039.py:10: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

fracked, treatment

  results['actions_c'] = model1.fit(cov_type='clustered', cluster_entity=True)


                          PanelOLS Estimation Summary                           
Dep. Variable:         lnactionnonoil   R-squared:                        0.0127
Estimator:                   PanelOLS   R-squared (Between):              0.2995
No. Observations:              143275   R-squared (Within):               0.0325
Date:                Sat, Sep 20 2025   R-squared (Overall):              0.1926
Time:                        18:57:18   Log-likelihood                -7.764e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      591.34
Entities:                        5731   P-value                           0.0000
Avg Obs:                       25.000   Distribution:                F(3,137517)
Min Obs:                       25.000                                           
Max Obs:                       25.000   F-statistic (robust):             63.735
                            

In [ ]:
# Store results for comparison
results = {}

# Model 1: actions_c - lnactionnonoil with controls
print("=== Model 1: actions_c ===")
model1 = PanelOLS(
    dependent=df_panel['lnactionnonoil'],
    exog=df_panel[['frack_post', 'fracked', 'treatment', 'lnestab', 'lnemp']],
    entity_effects=True,  # zipcode fixed effects
    time_effects=True,    # year fixed effects (i.year)
    drop_absorbed=True
)
results['actions_c'] = model1.fit(cov_type='clustered', cluster_entity=True)
print(results['actions_c'].summary)

# Model 2: facilities - lnone_non_oil without controls
print("\n=== Model 2: facilities ===")
model2 = PanelOLS(
    dependent=df_panel['lnone_non_oil'],
    exog=df_panel[['frack_post', 'fracked', 'treatment']],
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
)
results['facilities'] = model2.fit(cov_type='clustered', cluster_entity=True)
print(results['facilities'].summary)

# Model 3: facilities_c - lnone_non_oil with controls
print("\n=== Model 3: facilities_c ===")
model3 = PanelOLS(
    dependent=df_panel['lnone_non_oil'],
    exog=df_panel[['frack_post', 'fracked', 'treatment', 'lnestab', 'lnemp']],
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
)
results['facilities_c'] = model3.fit(cov_type='clustered', cluster_entity=True)
print(results['facilities_c'].summary)

# Model 4: formal - lnstate_formal with controls
print("\n=== Model 4: formal ===")
model4 = PanelOLS(
    dependent=df_panel['lnstate_formal_nonoil'],  # assuming this is the full variable name
    exog=df_panel[['frack_post', 'fracked', 'treatment', 'lnestab', 'lnemp']],
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
)
results['formal'] = model4.fit(cov_type='clustered', cluster_entity=True)
print(results['formal'].summary)

# Model 5: formal_c - same as formal (appears to be duplicate in your Stata code)
print("\n=== Model 5: formal_c ===")
model5 = PanelOLS(
    dependent=df_panel['lnstate_formal_nonoil'],
    exog=df_panel[['frack_post', 'fracked', 'treatment', 'lnestab', 'lnemp']],
    entity_effects=True,
    time_effects=True,
    drop_absorbed=True
)
results['formal_c'] = model5.fit(cov_type='clustered', cluster_entity=True)
print(results['formal_c'].summary)

# Create a summary comparison table (similar to Stata's esttab)
print("\n" + "="*80)
print("SUMMARY COMPARISON TABLE")
print("="*80)

# Extract key coefficients for comparison
models = ['actions_c', 'facilities', 'facilities_c', 'formal', 'formal_c']
variables = ['frack_post', 'fracked', 'treatment', 'lnestab', 'lnemp']

# Create comparison table
comparison_data = []
for var in variables:
    row = [var]
    for model in models:
        if var in results[model].params.index:
            coef = results[model].params[var]
            se = results[model].std_errors[var]
            pval = results[model].pvalues[var]
            
            # Add significance stars
            if pval < 0.01:
                stars = "***"
            elif pval < 0.05:
                stars = "**"
            elif pval < 0.1:
                stars = "*"
            else:
                stars = ""
            
            row.append(f"{coef:.4f}{stars}")
            row.append(f"({se:.4f})")
        else:
            row.extend(["-", "-"])
    comparison_data.append(row)

# Print comparison table
import pandas as pd
header = ['Variable']
for model in models:
    header.extend([model, 'SE'])

comp_df = pd.DataFrame(comparison_data)
comp_df.columns = header
print(comp_df.to_string(index=False))

print("\n" + "="*80)
print("MODEL STATISTICS")
print("="*80)

stats_data = []
for model in models:
    res = results[model]
    stats_data.append([
        model,
        f"{res.nobs}",
        f"{res.entity_info.total}",
        f"{res.rsquared:.4f}",
        f"{res.rsquared_within:.4f}"
    ])

stats_df = pd.DataFrame(stats_data, columns=['Model', 'N', 'Groups', 'R²', 'Within R²'])
print(stats_df.to_string(index=False))

print("\nNotes:")
print("* p<0.1, ** p<0.05, *** p<0.01")
print("Standard errors clustered by zipcode")
print("All models include zipcode and year fixed effects")

=== Model 1: actions_c ===


/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_53495/1163788316.py:21: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

fracked, treatment

  results['actions_c'] = model1.fit(cov_type='clustered', cluster_entity=True)


                          PanelOLS Estimation Summary                           
Dep. Variable:         lnactionnonoil   R-squared:                        0.0127
Estimator:                   PanelOLS   R-squared (Between):              0.2995
No. Observations:              143275   R-squared (Within):               0.0325
Date:                Sat, Sep 20 2025   R-squared (Overall):              0.1926
Time:                        18:46:07   Log-likelihood                -7.764e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      591.34
Entities:                        5731   P-value                           0.0000
Avg Obs:                       25.000   Distribution:                F(3,137517)
Min Obs:                       25.000                                           
Max Obs:                       25.000   F-statistic (robust):             63.735
                            

/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_53495/1163788316.py:33: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

fracked, treatment

  results['facilities'] = model2.fit(cov_type='clustered', cluster_entity=True)
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_53495/1163788316.py:45: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

fracked, treatment

  results['facilities_c'] = model3.fit(cov_type='clustered', cluster_entity=True)
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. D

                          PanelOLS Estimation Summary                           
Dep. Variable:          lnone_non_oil   R-squared:                        0.0100
Estimator:                   PanelOLS   R-squared (Between):              0.2837
No. Observations:              143275   R-squared (Within):               0.0245
Date:                Sat, Sep 20 2025   R-squared (Overall):              0.1803
Time:                        18:46:08   Log-likelihood                -4.036e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      463.85
Entities:                        5731   P-value                           0.0000
Avg Obs:                       25.000   Distribution:                F(3,137517)
Min Obs:                       25.000                                           
Max Obs:                       25.000   F-statistic (robust):             64.392
                            

/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_53495/1163788316.py:57: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

fracked, treatment

  results['formal'] = model4.fit(cov_type='clustered', cluster_entity=True)
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                            PanelOLS Estimation Summary                            
Dep. Variable:     lnstate_formal_nonoil   R-squared:                        0.0126
Estimator:                      PanelOLS   R-squared (Between):             -8.1354
No. Observations:                 143275   R-squared (Within):               0.0423
Date:                   Sat, Sep 20 2025   R-squared (Overall):             -2.6276
Time:                           18:46:08   Log-likelihood                -6.129e+04
Cov. Estimator:                Clustered                                           
                                           F-statistic:                      586.46
Entities:                           5731   P-value                           0.0000
Avg Obs:                          25.000   Distribution:                F(3,137517)
Min Obs:                          25.000                                           
Max Obs:                          25.000   F-statistic (robust):            

/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_53495/1163788316.py:69: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

fracked, treatment

  results['formal_c'] = model5.fit(cov_type='clustered', cluster_entity=True)


In [ ]:
import numpy as np
import doubleml as dml

In [4]:
!pip install doubleml

In [6]:
from doubleml.did.datasets import make_did_SZ2020

In [2]:
dml_data

In [5]:
df[df['id']== 0]

,id,y,y0,y1,d,t,Z1,Z2,Z3,Z4
0,0,214.771243,214.771243,214.105894,2025-04-01,2025-01-01,0.280397,-0.161563,0.374582,0.868365
1,0,219.717997,219.717997,219.651943,2025-04-01,2025-02-01,0.280397,-0.161563,0.374582,0.868365
2,0,225.510435,225.510435,223.591109,2025-04-01,2025-03-01,0.280397,-0.161563,0.374582,0.868365
3,0,232.354557,234.427969,232.354557,2025-04-01,2025-04-01,0.280397,-0.161563,0.374582,0.868365
4,0,238.340267,233.917183,238.340267,2025-04-01,2025-05-01,0.280397,-0.161563,0.374582,0.868365


In [1]:
import numpy as np
import doubleml as dml
from doubleml.did.datasets import make_did_CS2021
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
np.random.seed(42)
df = make_did_CS2021(n_obs=500)
dml_data = dml.data.DoubleMLPanelData(
    df,
    y_col="y",
    d_cols="d",
    id_col="id",
    t_col="t",
    x_cols=["Z1", "Z2", "Z3", "Z4"],
    datetime_unit="M"
)
ml_g = RandomForestRegressor(n_estimators=100, max_depth=5)
ml_m = RandomForestClassifier(n_estimators=100, max_depth=5)
dml_did_obj = dml.did.DoubleMLDIDMulti(
    obj_dml_data=dml_data,
    ml_g=ml_g,
    ml_m=ml_m,
    gt_combinations="standard",
    control_group="never_treated",
)
print(dml_did_obj.fit())

/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation

================== DoubleMLDIDMulti Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['Z1', 'Z2', 'Z3', 'Z4']
Instrument variable(s): None
Time variable: t
Id variable: id
No. Unique Ids: 500
No. Observations: 2500

------------------ Score & algorithm ------------------
Score function: observational
Control group: never_treated
Anticipation periods: 0

------------------ Machine learner   ------------------
Learner ml_g: RandomForestRegressor(max_depth=5)
Learner ml_m: RandomForestClassifier(max_depth=5)
Out-of-sample Performance:
Regression:
Learner ml_g0 RMSE: [[ 4.08196578  3.72787162  6.57623569 10.75409927  3.8236555   4.0543776
   3.79511693  7.67235656  3.61323481  3.650411    3.84079308  3.5520269 ]]
Learner ml_g1 RMSE: [[2.98466195 3.3130324  6.52477503 8.84005651 4.22669551 3.52880721
  3.73587004 7.07278738 3.91797397 3.78853269 3.66984001 4.19869841]]
Classification:
Learner ml_m

In [10]:
data['t'].unique()

array([0., 1.])

In [ ]:

#from doubleml.datasets import make_did_SZ2020
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
np.random.seed(42)
ml_g = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=5)
ml_m = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_leaf=5)
data = make_did_SZ2020(n_obs=500, cross_sectional_data=True, return_type='DataFrame')
obj_dml_data = dml.DoubleMLData(data, 'y', 'd', t_col='t')
dml_did_obj = dml.DoubleMLDIDCS(obj_dml_data, ml_g, ml_m)
dml_did_obj.fit().summary

/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/mcargnel/Documents/mea/tesis/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,coef,std err,t,P>|t|,2.5 %,97.5 %
d,-4.993875,7.561875,-0.660402,0.508996,-19.814876,9.827127
